In [1]:
import numpy as np
import pandas as pd
from flask import Flask, render_template, request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from webdriver_manager.chrome import ChromeDriverManager
import json
import bs4 as bs
import urllib.request
import pickle
import requests
import imdb
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep

import re
import pandas as pd
import os
import seaborn as sns
from selenium import webdriver 
import sys
import pandas as pd
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from matplotlib import pyplot as plt
import io
from collections import Counter
from nltk.corpus import wordnet
from scipy.stats import rankdata
from sklearn.linear_model import LinearRegression
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
import time
import concurrent.futures

warnings.filterwarnings("ignore")

In [2]:
imdb_movies=pd.DataFrame()
def webscrapping(content):
    for movie in content.select('.lister-item-content'):
        try: 
            actors=[a.text for a in movie.find('p',class_='').find_all('a')[1:]]
            data = {
                "title":movie.select('.lister-item-header')[0].get_text().strip(),
                "year":movie.select('.lister-item-year')[0].get_text().strip(),
#                 "certificate":movie.select('.certificate')[0].get_text().strip(),
                "time":movie.select('.runtime')[0].get_text().strip(),
                "genre":movie.select('.genre')[0].get_text().strip(),
                "rating":movie.select('.ratings-imdb-rating')[0].get_text().strip(),
#                 "metascore":movie.select('.ratings-metascore')[0].get_text().strip(),
                "simple_desc":movie.select('.text-muted')[2].get_text().strip(),
                "votes":movie.select('.sort-num_votes-visible')[0].get_text().strip(),
                "director":movie.find('p',class_='').find_all('a')[0].text,
                "Cast":actors

            }
            movie_list.append(data)
        except IndexError:
            continue
    print(len(movie_list),'\n')
    return movie_list

In [3]:
# imdb_movies.to_csv('imdb_movies_allgenres.csv')        
# print(imdb_movies.shape)

# url_dict = {}
# HEADERS ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
# movie_list=[]
# for year in range(2000,2022):
#     threads = min(50, year)
#     for i in range (1,5000,50):
#         url = "https://www.imdb.com/search/title/?title_type=feature&start={}&release_date={}&sort=num_votes,desc&ref_=adv_nxt"
#         url = url.format(i,year)
#         resp = requests.get(url, headers=HEADERS)
#         content = BeautifulSoup(resp.content, 'lxml')
#         webscrapping(content)

#         #         with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
# #             executor.map(webscrapping,content)
            
#         print(url)

In [4]:
data = pd.read_csv('imdb_movies_allgenres.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
movies_titles=data['title'].str.split("\n",expand=True)
data['title'] = movies_titles[1]
data['comb']=data['director']+data['Cast']+data['genre']
data['votes']=data['votes'].str.split("\n",expand=True)[1]
data["votes"] = [float(str(i).replace(",", "")) for i in data["votes"]]

data['title'].nunique()

data=data.head(20000)

data

movie = input("Movie_Name")

data

data['title'].nunique()

Movie_NameAmerican Psycho


19547

In [5]:
filename = 'nlp_model.pkl'
with open(filename, 'rb') as f:
    clf = pickle.load(f)

with open('tranform.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

def create_similarity():
    # creating a count matrix
    cv = TfidfVectorizer()
    count_matrix = cv.fit_transform(data['comb'])
    # creating a similarity score matrix
    similarity = cosine_similarity(count_matrix)
    return similarity

def rcmd(m):
    m = m.lower()
    similarity = create_similarity()
    if m not in data['title'].str.lower().unique():
        return('Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies')
    else:
            
            i = data.loc[data['title'].str.lower()==m.lower()].index[0]
            lst = list(enumerate(similarity[min(len(similarity)-1,i)]))
            lst = sorted(lst, key = lambda x:x[1] ,reverse=True)
            lst = lst[1:11] # excluding first item since it is the requested movie itself
            l = []
            
            for i in range(len(lst)):
                try:
                    a = lst[i][0]
                    l.append(data['title'][a])
                    print(l)
                except :
                    continue
            
    return l
    
def convert_to_list(my_list):
    my_list = my_list.split('","')
    my_list[0] = my_list[0].replace('["','')
    my_list[-1] = my_list[-1].replace('"]','')
    return my_list

def get_suggestions(df):
    return list(df['title'].str.capitalize())

def home(df):
    suggestions = get_suggestions(df)
    return suggestions

def similarity(movie):
#     print("in similarity")
    rc = rcmd(movie)
    if type(rc)==type('string'):
        return rc
    else:
        m_str="---".join(rc)
        return rc
    
def recommend(movie):
    # creating instance of IMDb
    ia = imdb.IMDb()
    # searching the name
    search = ia.search_movie(movie)
    # loop for printing the name and id
    for i in range(len(search)):
        if (search[i]['title'].lower() == movie.lower()):
            imdb_id = 'tt'+str(search[i].movieID)
            break

    # web scraping to get user reviews from IMDB site
    sys.path.insert(0,'/usr/local/bin/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')


    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
    url = 'https://www.imdb.com/title/{}/reviews?ref_=tt_urv'.format(imdb_id)
    print(url)
    driver.get(url)
    print(driver.title)
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    review_date_list = []
    review_title_list = []
    author_list = []
    review_list = []
    review_url_list = []
    error_url_list = []
    error_msg_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

    for d in tqdm(reviews):
        try:
            sel2 = Selector(text = d.get_attribute('innerHTML'))
            try:
                review = sel2.css('.text.show-more__control::text').extract_first()
            except:
                review = np.NaN
            try:
                review_date = sel2.css('.review-date::text').extract_first()
            except:
                review_date = np.NaN    
            try:
                author = sel2.css('.display-name-link a::text').extract_first()
            except:
                author = np.NaN    
            try:
                review_title = sel2.css('a.title::text').extract_first()
            except:
                review_title = np.NaN
            try:
                review_url = sel2.css('a.title::attr(href)').extract_first()
            except:
                review_url = np.NaN
            review_date_list.append(review_date)
            review_title_list.append(review_title)
            author_list.append(author)
            review_list.append(review)
            review_url_list.append(review_url)
        except Exception as e:
            error_url_list.append(url)
            error_msg_list.append(e)
    review_df = pd.DataFrame({
        'Review_Date':review_date_list,
        'Author':author_list,
        'Review_Title':review_title_list,
        'Review':review_list,
        'Review_Url':review_url
        })
    review_df
    reviews_status=[]
    reviews_list =[]
    for reviews in review_df['Review']:
        if (reviews is not None):
            reviews_list.append(reviews)
            # passing the review to our model
            movie_review_list = np.array([reviews])
            movie_vector = vectorizer.transform(movie_review_list)
            pred = clf.predict(movie_vector)
            reviews_status.append('Good' if pred else 'Bad')

    # combining reviews and comments into a dictionary
            movie_reviews = {reviews_list[i]: reviews_status[i] for i in range(len(reviews_list))}  
            movie_reviews_df = pd.DataFrame(movie_reviews.items(),columns={'Review','Outcome'})
    return (movie_reviews_df)

movie_list=similarity(movie)   

movie

['The Notorious Bettie Page']
['The Notorious Bettie Page', 'Mulholland Drive']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes', 'Happy End']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes', 'Happy End', 'Shaft']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes', 'Happy End', 'Shaft', 'Batman Begins']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes', 'Happy End', 'Shaft', 'Batman Begins', 'EvenHand']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes', 'Happy End', 'Shaft', 'Batman Begins', 'EvenHand', '3:10 to Yuma']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes', 'Happy End', 'Shaft', 'Batman Begins', 'EvenHand', '3:10 to Yuma', 'Harsh Times']
['The Notorious Bettie Page', 'Mulholland Drive', 'The Legend of Lucy Keyes', 'Happy End', 'Shaft', 

'American Psycho'

In [7]:
f_Df=recommend(movie)
score = ((f_Df.Outcome.value_counts()[0])/len(f_Df))
score

df=pd.DataFrame()
for i in range(0,len(movie_list)):
    a=data[data['title']==movie_list[i]]
    df=pd.concat([a,df])
df['Recommendation_Score'] = 0

https://www.imdb.com/title/tt0144084/reviews?ref_=tt_urv
American Psycho (2000) - American Psycho (2000) - User Reviews - IMDb


100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 152.36it/s]


In [8]:
df

,title,year,time,genre,rating,simple_desc,votes,director,Cast,comb,Recommendation_Score
2435,Captain Corelli's Mandolin,(2001),131 min,"Drama, Music, Romance",5.9,When a fisherman leaves to fight with the Gree...,34624.0,John Madden,"['Nicolas Cage', 'Penélope Cruz', 'John Hurt',...","John Madden['Nicolas Cage', 'Penélope Cruz', '...",0
12962,Harsh Times,(2005),116 min,"Action, Crime, Drama",6.8,A tough-minded drama about two friends in Sout...,66155.0,David Ayer,"['Christian Bale', 'Freddy Rodríguez', 'Eva Lo...","David Ayer['Christian Bale', 'Freddy Rodríguez...",0
19679,3:10 to Yuma,(2007),122 min,"Action, Crime, Drama",7.7,A small-time rancher agrees to hold a captured...,311684.0,James Mangold,"['Russell Crowe', 'Christian Bale', 'Ben Foste...","James Mangold['Russell Crowe', 'Christian Bale...",0
5665,EvenHand,(2002),92 min,Drama,7.3,Set in a dirt-poor neighborhood in the fiction...,478.0,Joseph Pierson,"['Bill Sage', 'Bill Dawes', 'Lawrence Stringer...","Joseph Pierson['Bill Sage', 'Bill Dawes', 'Law...",0
12862,Batman Begins,(2005),140 min,"Action, Crime, Drama",8.2,"After training with his mentor, Batman begins ...",1457772.0,Christopher Nolan,"['Christian Bale', 'Michael Caine', 'Ken Watan...","Christopher Nolan['Christian Bale', 'Michael C...",0
61,Shaft,(2000),99 min,"Action, Crime, Thriller",6.0,New York City police detective John Shaft (nep...,77514.0,John Singleton,"['Samuel L. Jackson', 'Vanessa Williams', 'Chr...","John Singleton['Samuel L. Jackson', 'Vanessa W...",0
7801,Happy End,(2003),98 min,"Comedy, Romance, Drama",5.3,A young French woman comes to NYC to be an act...,1479.0,Amos Kollek,"['Audrey Tautou', 'Justin Theroux', 'Jennifer ...","Amos Kollek['Audrey Tautou', 'Justin Theroux',...",0
19299,Happy End,(2006),122 min,Drama,5.1,Add a Plot,15.0,Radu Potcoavã,"['George Alexandru', 'Dan Badarau', 'Constanti...","Radu Potcoavã['George Alexandru', 'Dan Badarau...",0
16761,The Legend of Lucy Keyes,(2006),93 min,"Drama, Thriller",4.7,An urban family leaves city life behind for th...,1574.0,John Stimpson,"['Julie Delpy', 'Justin Theroux', 'Brooke Adam...","John Stimpson['Julie Delpy', 'Justin Theroux',...",0
2324,Mulholland Drive,(2001),147 min,"Drama, Mystery, Thriller",7.9,After a car wreck on the winding Mulholland Dr...,354386.0,David Lynch,"['Naomi Watts', 'Laura Harring', 'Justin Thero...","David Lynch['Naomi Watts', 'Laura Harring', 'J...",0


In [9]:
for i in range(0,len(df)):
#     print(df['rating'].iloc[i], df['votes'].iloc[i])
    df['Recommendation_Score'].iloc[i]= (df['votes'].iloc[i]/df['rating'].iloc[i])*score
    

In [10]:
df.sort_values(by=['Recommendation_Score'],ascending=False,inplace=True)

print("According to the title given for search : " ,movie)


df[['title','rating','Cast','genre']]

According to the title given for search :  American Psycho


,title,rating,Cast,genre
12862,Batman Begins,8.2,"['Christian Bale', 'Michael Caine', 'Ken Watan...","Action, Crime, Drama"
2324,Mulholland Drive,7.9,"['Naomi Watts', 'Laura Harring', 'Justin Thero...","Drama, Mystery, Thriller"
19679,3:10 to Yuma,7.7,"['Russell Crowe', 'Christian Bale', 'Ben Foste...","Action, Crime, Drama"
61,Shaft,6.0,"['Samuel L. Jackson', 'Vanessa Williams', 'Chr...","Action, Crime, Thriller"
12962,Harsh Times,6.8,"['Christian Bale', 'Freddy Rodríguez', 'Eva Lo...","Action, Crime, Drama"
2435,Captain Corelli's Mandolin,5.9,"['Nicolas Cage', 'Penélope Cruz', 'John Hurt',...","Drama, Music, Romance"
13108,The Notorious Bettie Page,6.5,"['Gretchen Mol', 'Lili Taylor', 'Chris Bauer',...","Biography, Drama"
16761,The Legend of Lucy Keyes,4.7,"['Julie Delpy', 'Justin Theroux', 'Brooke Adam...","Drama, Thriller"
7801,Happy End,5.3,"['Audrey Tautou', 'Justin Theroux', 'Jennifer ...","Comedy, Romance, Drama"
5665,EvenHand,7.3,"['Bill Sage', 'Bill Dawes', 'Lawrence Stringer...",Drama


In [15]:
underrated_movie = df[df['votes'] < df['votes'].median()].sort_values(by='rating', ascending = True).iloc[0]['title']
print("If you liked "+movie+" then you might also like "+underrated_movie)

If you liked American Psycho then you might also like The Legend of Lucy Keyes
